<a href="https://colab.research.google.com/github/lilasch/Facial_Attribute_CNN/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Collection and Training

In [ ]:
import tensorflow
import tensorflow.keras as keras
import pandas as pd
import numpy as np
import sklearn
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Input, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.metrics import classification_report, multilabel_confusion_matrix


# Load attributes csv
attributes = pd.read_csv("list_attr_celeba.txt", skiprows = 1, delimiter="\s+|\t")

# Load csv with partitions values
partitions = pd.read_csv("list_eval_partition.txt", delimiter="\s+|\t", header = None)

partitions.columns = ['image_name', 'dataset'] # setting column header names for partitions
attributes['dataset'] = partitions['dataset'].values # copying the partition values into the attributes df

# unzipping image data
import zipfile
zippath = 'img_align_celeba.zip'
targetfolder = 'all_images'

with zipfile.ZipFile(zippath, 'r') as zip_ref:
    zip_ref.extractall(targetfolder)

# the image filenames need to be in their own column called "image_names"
attributes = attributes.reset_index()
attributes.rename(columns={'index': 'image_names'}, inplace=True)

# splitting the data
train = attributes[attributes['dataset'] == 0]
train.drop(columns='dataset', inplace=True)
train = train.iloc[:1000]
valid = attributes[attributes['dataset'] == 1]
valid.drop(columns='dataset', inplace=True)
valid = valid.iloc[:1000]
test = attributes[attributes['dataset'] == 2]
test.drop(columns='dataset', inplace=True)
test = test.iloc[:1000]

batch_size = 64

attribute_names = ['5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair',
'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
                    'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling',
                    'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']

# replace all -1s with 0s for binary classification
for name in attribute_names:
  train[name].replace({-1: 0}, inplace=True)
  valid[name].replace({-1: 0}, inplace=True)
  test[name].replace({-1: 0}, inplace=True)

# Use ImageDataGenerator for train and valid datasets
train_datagen = ImageDataGenerator(rescale=1.0/255)
valid_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow from dataframe for train and valid generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train,
    directory='all_images/img_align_celeba/',
    x_col="image_names",
    y_col=attribute_names,
    class_mode="raw",
    batch_size=batch_size,
    target_size=(109,89)
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=valid,
    directory='all_images/img_align_celeba/',
    x_col="image_names",
    y_col=attribute_names,
    class_mode="raw",
    batch_size=batch_size,
    target_size=(109,89)
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory='all_images/img_align_celeba/',
    x_col="image_names",
    y_col=attribute_names,
    class_mode="raw",
    batch_size=batch_size,
    target_size=(109,89)
)


#Initial Sequential Model

In [ ]:
model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                 input_shape=(109, 89, 3)))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Flatten())
model.add(Dense(units=32, activation="relu"))
model.add(Dense(units=40, activation="sigmoid"))
model.add(Dropout(0.1))
model.summary()

# Compile and train your model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.001,
    monitor = 'accuracy',
    restore_best_weights=True
)

model.fit_generator(generator=train_generator,
                    epochs=1000,
                    validation_data=valid_generator,
                    verbose=1,
                    callbacks = [early_stopping])


#Pre-Trained model

In [ ]:
input_shape = (109, 89, 3)
input_layer = Input(shape=input_shape)
resized_input = Resizing(224, 224)(input_layer) # resize images to 224x224 to fit ImageNet

# Load the pre-trained ResNet50 model (excluding the top classification layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=resized_input)

# Add custom layers on top of ResNet50
x = base_model(base_model.input, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(64, activation='relu', kernel_initializer=initializers.glorot_uniform())(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
predictions = Dense(40, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base ResNet50 layetrainrs
for layer in base_model.layers:
    layer.trainable = False

# Compile and train the model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'binary_accuracy'])

# Define early stopping after 5 epochs
early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.001,
    monitor = 'accuracy',
    restore_best_weights=True
)

# Train on the training data for 10 epochs
model.fit(train_generator,
                    epochs=10,
                    validation_data=valid_generator,
                   steps_per_epoch=train.shape[0]//batch_size,
                    validation_steps=valid.shape[0]//batch_size,
                    verbose=1,
                    callbacks = [early_stopping])

# Fine-tune the model
for layer in model.layers[25:]:
    layer.trainable = True

# Recompile the model after fine-tuning
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Continue training with fine-tuning
model.fit(train_generator, validation_data=valid_generator, steps_per_epoch=train.shape[0]//batch_size, epochs=10, callbacks = [early_stopping])

#Evaluation

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
for name in attribute_names:
  test[name].replace({-1: 0}, inplace=True)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test,
    directory='all_images/img_align_celeba/',
    x_col="image_names",
    y_col=attribute_names,
    class_mode="raw",
    batch_size=batch_size,
    target_size=(109,89)
)

predictions = model.predict(test_generator)
predictions_rounded = np.round(predictions)
true_labels = test_generator.labels
true_labels_binary = (true_labels == 1).astype(int)
true_labels_binary_reshaped = true_labels_binary.reshape(predictions_rounded.shape)


print(classification_report(true_labels_binary_reshaped, predictions_rounded, target_names=attribute_names))

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np

# Define an empty array to store the predictions and true labels for all batches from the test generator
all_predictions = []
all_true_labels = []

# Loop through the test generator to obtain predictions and true labels for all batches
test_generator.reset()  # Reset the test generator to starttrue_labels = test_generator.labels from the beginning
for i in range(len(test_generator)):       # run for however many batches we want, I started with two because it took a while
    batch_data, batch_labels = test_generator.next()
    batch_predictions = model.predict(batch_data)
    all_predictions.append(batch_predictions)
    all_true_labels.append(batch_labels)


# Concatenate the predictions and true labels from all batches into single arrays
predictions = np.concatenate(all_predictions)
true_labels = np.concatenate(all_true_labels)

# Convert to binary
true_labels_binary = (true_labels == 1).astype(int)
predictions_binary = (predictions > 0.5).astype(int)

# Generate the multilabel confusion matrix
confusion_matrix = multilabel_confusion_matrix(true_labels_binary, predictions_binary)  # threshold

# Reshape and print the multilabel confusion matrix
confusion_matrix = confusion_matrix.reshape(40, 2, 2)
print(confusion_matrix)

In [ ]:
fig, axes = plt.subplots(10, 4, figsize=(20, 40))  # Create a 10x4 grid of subplots
for i in range(40):
    ax = axes[i // 4, i % 4]  # Get the current subplot
    sns.heatmap(confusion_matrix[i], annot=True, cmap='Blues', fmt='d', cbar=False, ax=ax)
    ax.set_xlabel('Predicted Label')
    ax.set_ylabel('True Label')
    ax.set_title(attribute_names[i])
plt.tight_layout()
plt.show()

In [ ]:
def predict_single_image(model, img_path, target_size):
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=target_size)
    plt.imshow(img)
    plt.show()
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create a batch with a single sample
    img_array /= 255.0  # Normalize pixel values

    # Make predictions
    predictions = model.predict(img_array)
    return predictions

img_path = '/content/all_images/img_align_celeba/000008.jpg'
target_size = (109, 89)  # Specify the target size used during training

predictions = predict_single_image(model, img_path, target_size)
# Print predictions in a more readable way
for i, prob in enumerate(predictions[0]):
    print(f"Probability of {attribute_names[i]}: {prob:.4f}")

In [ ]:
def predict_real_picture(img_path):
    img = image.load_img(img_path)
    plt.imshow(img)
    plt.show()
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create a batch with a single sample
    img_array /= 255.0  # Normalize pixel values

    # Make predictions
    predictions = model.predict(img_array)
    for i, prob in enumerate(predictions[0]):
      print(f"Probability of {attribute_names[i]}: {prob:.4f}")

predict_real_picture("/content/drive/MyDrive/selfie.png")
predict_real_picture("/content/drive/MyDrive/sam.jpg")